In [1]:
import os
import kagglehub
import shutil
os.chdir('../')

/home/akill_sud/anaconda3/envs/cnncls/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%pwd

'/home/akill_sud/Documents/projects/Cauliflower_Disease_Prediction'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from Cauliflower_Classifier.constants import *
from Cauliflower_Classifier.utils.common import read_yaml_file,create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml_file(config_filepath)
        self.params = read_yaml_file(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [6]:
import os
import zipfile
import gdown
from  Cauliflower_Classifier import logger
from Cauliflower_Classifier.utils.common import get_size

In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[3]
            prefix = 'https://drive.google.com/uc?export=download&id=14vzgSzyJXadU967e8ilb7ByxGoDXh5DZ'
            gdown.download(prefix,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)





In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-11-07 20:36:26,241 INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-11-07 20:36:26,247 INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-07 20:36:26,249 INFO: common: Created directory at : artifacts]
[2024-11-07 20:36:26,250 INFO: common: Created directory at : artifacts/data_ingestion]
[2024-11-07 20:36:26,251 INFO: 3626262633: Downloading data from https://drive.google.com/file/d/14vzgSzyJXadU967e8ilb7ByxGoDXh5DZ/view?usp=sharing into file artifacts/data_ingestion/dataset.zip]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=14vzgSzyJXadU967e8ilb7ByxGoDXh5DZ
From (redirected): https://drive.google.com/uc?export=download&id=14vzgSzyJXadU967e8ilb7ByxGoDXh5DZ&confirm=t&uuid=b29595a8-3644-4eb2-b6e0-256b0a68f2d3
To: /home/akill_sud/Documents/projects/Cauliflower_Disease_Prediction/artifacts/data_ingestion/dataset.zip
100%|██████████| 1.55G/1.55G [04:20<00:00, 5.96MB/s]

[2024-11-07 20:40:51,865 INFO: 3626262633: Downloaded data from https://drive.google.com/file/d/14vzgSzyJXadU967e8ilb7ByxGoDXh5DZ/view?usp=sharing into file artifacts/data_ingestion/dataset.zip]
